In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# Test
from Transformers.DataLoader import Eng2Ita

loader = Eng2Ita(path="eng-ita/eng-ita.txt", max_tokens_source=15000, max_tokens_target=20000, seq_length_source=20, seq_length_target=20)

Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB



2023-05-26 15:13:01.255762: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-26 15:13:01.255889: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
train_dataset, val_dataset, test_dataset = loader(batch_size=5)

read_file... (1/4)
split data... (2/4)
tokenize data... (3/4


2023-05-26 15:13:02.129723: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-26 15:13:02.176903: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-26 15:13:29.821842: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


building datasets... (4/4)
Done!


In [4]:
next(iter(train_dataset))[0]["source"][0].numpy()

array([  41,  635,  130,   30,  348,   19,    7, 9508,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [5]:
loader.source_tokenizer.get_vocabulary()[:10]

['', '[UNK]', 'tom', 'you', 'i', 'to', 'the', 'a', 'is', 'do']

In [6]:
loader.target_tokenizer.get_vocabulary()[:10]

['', '[UNK]', '[start]', '[end]', 'tom', 'non', 'di', 'a', 'è', 'che']

In [7]:
loader.inspect_from_dataset(train_dataset)

source:  i have plans for saturday night              
target:  [start] io ho dei piani per sabato sera [end]           
source:  tom wanted to be happy again              
target:  [start] tom voleva essere di nuovo felice [end]            
source:  it made toms family really happy              
target:  [start] ha reso la famiglia di tom davvero felice [end]          
source:  i dont want meat                
target:  [start] non voglio carne [end]               
source:  some people walk to work               
target:  [start] alcune persone vanno al lavoro a piedi [end]           
source:  george washington arrived the day before              
target:  [start] george washington arrivò il giorno prima [end]            
source:  his eyes look like those of a leopard            
target:  [start] i suoi occhi sembrano quelli di un leopardo [end]          
source:  ive thought about it recently               
target:  [start] ci ho pensato recentemente [end]              
source:  i nev

In [8]:
# Test encoder decoder

from Transformers.PositionalEncoding import BasicPositionalEmbeddings
from Transformers.Encoder import TransformerEncoder
from Transformers.Decoder import TransformerDecoder


In [9]:
# tf.keras.backend.clear_session()

# num_heads = 8
# DENSE_DIM = 2048

# encoder_inputs = tf.keras.Input(shape=(None, ), dtype="int64", name="source")

# x = transformers.PositionaEmbeddings(dim_emb=EMBEDDING_SIZE, max_tokens=MAX_TOKENS, max_seq_lenght=MAX_SEQ)(encoder_inputs)

# encoder_outputs = transformers.TransformerEncoder(num_heads=num_heads, dim_emb=EMBEDDING_SIZE, dim_dense=DENSE_DIM)(x)

# decoder_inputs = tf.keras.Input(shape=(None, ), dtype="int64", name="target")

# x = transformers.PositionaEmbeddings(dim_emb=EMBEDDING_SIZE, max_tokens=MAX_TOKENS, max_seq_lenght=MAX_SEQ)(decoder_inputs)

# #Source, target
# x = TransformerDecoder(num_heads=num_heads, dim_dense=DENSE_DIM, dim_emb=EMBEDDING_SIZE)(encoder_outputs, x)

# x = tf.keras.layers.Dropout(0.5)(x)

# decoder_output = tf.keras.layers.Dense(MAX_TOKENS, activation="softmax")(x)

# transformer_eng_spa_model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_output) 

In [10]:
tf.keras.backend.clear_session()

In [11]:
source = tf.keras.Input(shape=(None, ), dtype="int64", name="source")
source_positional_embeddings = PositionaEmbeddings(dim_emb=512, max_tokens=15000, max_seq_length=20)(source)
source_encoded_sequence = TransformerEncoder(num_heads=8, dim_dense=1024, dim_emb=512)(source_positional_embeddings)

target = tf.keras.Input(shape=(None, ), dtype="int64", name="target")
target_positional_embeddings = PositionaEmbeddings(dim_emb=512, max_tokens=20000, max_seq_length=20)(target)
decoded_sequence = TransformerDecoder(num_heads=8, dim_dense=1024, dim_emb=512)(source_encoded_sequence, target_positional_embeddings)

decoder_sequence_dropout = tf.keras.layers.Dropout(0.5)(decoded_sequence)

next_token = tf.keras.layers.Dense(units=20000, activation="softmax")(decoder_sequence_dropout)

NameError: name 'PositionaEmbeddings' is not defined

In [ ]:
model = tf.keras.Model(inputs=[source, target], outputs=next_token)

In [ ]:
model.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
# model.fit(train_dataset, val_dataset=val_dataset, epochs=5)

In [12]:
from Transformers.Seq2SeqModels import TransformerBasedTranslator

In [13]:
translator_model = TransformerBasedTranslator(num_heads=3, dim_embeddings=256, dim_dense=1024, source_max_seq_length=15000, target_max_seq_length=30000, source_max_tokens=20, target_max_tokens=20, dropout_rate=0.4)

In [14]:
translator_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-3), metrics=["accuracy"])

In [15]:
train_dataset, val_dataset, test_dataset = Eng2Ita(path="eng-ita/eng-ita.txt", max_tokens_source=15000, max_tokens_target=30000, seq_length_source=20, seq_length_target=20)(batch_size=512)

read_file... (1/4)
split data... (2/4)
tokenize data... (3/4


2023-05-26 15:13:53.227812: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-26 15:14:21.181137: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


building datasets... (4/4)
Done!


In [17]:
from Transformers.ModelBuilder import TransformerTranslator

TransformerTranslator(path="eng-ita/eng-ita.txt", num_heads=3, dim_dense=1024, dim_embeddings=256, source_max_tokens=15000, target_max_tokens=30000, source_seq_length=20, target_seq_length=20, dropout_rate=0.4).fit(epochs=5, batch_size=512, output_model_path="test_model")


ImportError: cannot import name 'Source2Target' from 'Transformers.DataLoader' (/Users/matiasgonzalez/workspace/eng2pasta/Transformers/DataLoader.py)

In [16]:
# translator_model.fit(train_dataset, validation_data=val_dataset, epochs=5)

Epoch 1/5


2023-05-26 15:14:32.233102: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


534/534 [==============================] - ETA: 0s - loss: 0.4891 - accuracy: 0.7091

2023-05-26 15:25:09.320364: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


534/534 [==============================] - 691s 1s/step - loss: 0.4891 - accuracy: 0.7091 - val_loss: 0.2969 - val_accuracy: 0.7351
Epoch 2/5
534/534 [==============================] - 660s 1s/step - loss: 0.6347 - accuracy: 0.7197 - val_loss: 0.3519 - val_accuracy: 0.7330
Epoch 3/5
445/534 [========================>.....] - ETA: 1:41 - loss: 0.8927 - accuracy: 0.7166

KeyboardInterrupt: 